# Document AI Specialized Parser (Async)
This notebook shows you how use Document AI's specialized parsers ex. Invoice, Receipt, and W9 asynchronously.

In [ ]:
# Install necessary Python libraries and restart your kernel after.
!python -m pip install -r ../requirements.txt

In [ ]:
from google.cloud import documentai_v1beta3 as documentai
from google.cloud import storage

import re
import os
import pandas as pd
import simplejson as json

## Set your processor variables 

In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID_HERE"
LOCATION = "us"  # Format is 'us' or 'eu'
PROCESSOR_ID = "PROCESSOR_ID"  # Create processor in Cloud Console

GCS_INPUT_BUCKET = 'cloud-samples-data'
GCS_INPUT_PREFIX = 'documentai/async_invoices/'
GCS_OUTPUT_URI = 'YOUR-OUTPUT-BUCKET'
GCS_OUTPUT_URI_PREFIX = 'TEST'
TIMEOUT = 300

The following code calls the synchronous API and parses the form fields and values.

In [ ]:
def process_document_sample():
    # Instantiates a client
    client_options = {"api_endpoint": "{}-documentai.googleapis.com".format(LOCATION)}
    client = documentai.DocumentProcessorServiceClient(client_options=client_options)
    storage_client = storage.Client()
    
    # Sample invoices are stored in gs://cloud-samples-data/documentai/async_invoices/
    blobs = storage_client.list_blobs(GCS_INPUT_BUCKET, prefix=GCS_INPUT_PREFIX)
    input_configs = []
    print("Input Files:")
    for blob in blobs:
        if ".pdf" in blob.name:
            source = "gs://{bucket}/{name}".format(bucket = GCS_INPUT_BUCKET, name = blob.name)
            print(source)
            input_config = documentai.types.document_processor_service.BatchProcessRequest.BatchInputConfig(
                gcs_source=source, mime_type="application/pdf")
            input_configs.append(input_config)

    destination_uri = f"{GCS_OUTPUT_URI}/{GCS_OUTPUT_URI_PREFIX}/"

    # Where to write results
    output_config = documentai.types.document_processor_service.BatchProcessRequest.BatchOutputConfig(
        gcs_destination=destination_uri
    )

    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id
    # You must create new processors in the Cloud Console first.
    name = f"projects/{PROJECT_ID}/locations/{LOCATION}/processors/{PROCESSOR_ID}"
    request = documentai.types.document_processor_service.BatchProcessRequest(
        name=name,
        input_configs=input_configs,
        output_config=output_config,
    )

    operation = client.batch_process_documents(request)

    # Wait for the operation to finish
    operation.result(timeout=TIMEOUT)

    # Results are written to GCS. Use a regex to find
    # output files
    match = re.match(r"gs://([^/]+)/(.+)", destination_uri)
    output_bucket = match.group(1)
    prefix = match.group(2)

    bucket = storage_client.get_bucket(output_bucket)
    blob_list = list(bucket.list_blobs(prefix=prefix))

    for i, blob in enumerate(blob_list):
        # If JSON file, download the contents of this blob as a bytes object.
        if ".json" in blob.name:
            blob_as_bytes = blob.download_as_string()
            print("downloaded")

            document = documentai.types.Document.from_json(blob_as_bytes)
            print(f"Fetched file {i + 1}")

            # For a full list of Document object attributes, please reference this page:
            # https://cloud.google.com/document-ai/docs/reference/rpc/google.cloud.documentai.v1beta3#document
            
            # Read the entities output from the processor
            types = []
            values = []
            confidence = []
            
            for entity in document.entities:
                types.append(entity.type_)
                values.append(entity.mention_text)
                confidence.append(round(entity.confidence,4))
        
            # Create a Pandas Dataframe to print the values in tabular format. 
            df = pd.DataFrame({'Type': types, 'Value': values, 'Confidence': confidence}) 
            display(df)
                
        else:
            print(f"Skipping non-supported file type {blob.name}")


# Extract shards from the text field
def get_text(doc_element: dict, document: dict):
    """
    Document AI identifies form fields by their offsets
    in document text. This function converts offsets
    to text snippets.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in doc_element.text_anchor.text_segments:
        start_index = (
            int(segment.start_index)
            if segment in doc_element.text_anchor.text_segments
            else 0
        )
        end_index = int(segment.end_index)
        response += document.text[start_index:end_index]
    return response

In [ ]:
doc = process_document_sample()